In [1]:
import pandas as pd

# Load the data
file_path = 'ChestnutRidge_laptops.xlsx'  # Make sure the file path is correct
data = pd.read_excel(file_path)

# Display the first few rows of the data
data.head()

,Run,Participant ID,8GB_RAM,16GB_RAM,14_inch_screen,15_inch_screen,8_hour_battery,10_hour_battery,price_850,price_900,SATA_HardDrive,Rating
0,1,1,0,0,1,0,0,1,0,0,0,9
1,2,1,1,0,0,1,1,0,0,1,1,8
2,3,1,1,0,1,0,0,0,0,0,1,9
3,4,1,0,1,0,1,0,0,0,0,0,10
4,5,1,1,0,1,0,1,0,0,1,0,6


In [3]:
# Check for missing values
missing_values = data.isnull().sum()

# Check data types
data_types = data.dtypes

# Print the results
print("Missing values per column:\n", missing_values)
print("\nData types of each column:\n", data_types)

Missing values per column:
 Run                0
Participant ID     0
8GB_RAM            0
16GB_RAM           0
14_inch_screen     0
15_inch_screen     0
8_hour_battery     0
10_hour_battery    0
price_850          0
price_900          0
SATA_HardDrive     0
Rating             0
dtype: int64

Data types of each column:
 Run                int64
Participant ID     int64
8GB_RAM            int64
16GB_RAM           int64
14_inch_screen     int64
15_inch_screen     int64
8_hour_battery     int64
10_hour_battery    int64
price_850          int64
price_900          int64
SATA_HardDrive     int64
Rating             int64
dtype: object


In [5]:
# Step 3: Identify the baseline laptop configuration
# Filter the data to find the rows where all dummy-coded attributes are 0
baseline = data[(data['8GB_RAM'] == 0) & 
                (data['16GB_RAM'] == 0) &
                (data['14_inch_screen'] == 0) &
                (data['15_inch_screen'] == 0) &
                (data['8_hour_battery'] == 0) &
                (data['10_hour_battery'] == 0) &
                (data['price_850'] == 0) &
                (data['price_900'] == 0) &
                (data['SATA_HardDrive'] == 0)]

# Display the baseline configuration
baseline

,Run,Participant ID,8GB_RAM,16GB_RAM,14_inch_screen,15_inch_screen,8_hour_battery,10_hour_battery,price_850,price_900,SATA_HardDrive,Rating


In [7]:
# Step 4: Calculate Average Partworths for Each Attribute Level
# Group the data by each attribute level and calculate the mean rating across participants

# Group by dummy-coded attributes and calculate the average rating
partworths = data.groupby(['8GB_RAM', '16GB_RAM', '14_inch_screen', '15_inch_screen',
                           '8_hour_battery', '10_hour_battery', 'price_850', 'price_900', 'SATA_HardDrive']).Rating.mean()

# Reset the index for better readability
partworths = partworths.reset_index()

# Display the calculated partworths
partworths

,8GB_RAM,16GB_RAM,14_inch_screen,15_inch_screen,8_hour_battery,10_hour_battery,price_850,price_900,SATA_HardDrive,Rating
0,0,0,0,0,0,0,0,1,0,5.85
1,0,0,0,0,1,0,1,0,0,8.75
2,0,0,0,1,1,0,0,0,1,8.85
3,0,0,1,0,0,1,0,0,0,9.25
4,0,0,1,0,0,1,1,0,1,7.60
5,0,1,0,0,0,1,0,1,1,5.15
6,0,1,0,0,1,0,0,0,1,7.95
7,0,1,0,1,0,0,0,0,0,9.25
8,0,1,1,0,1,0,1,0,0,6.65
9,1,0,0,0,0,0,1,0,1,7.20


In [9]:
# Step 5: Calculate the importance of each attribute based on the range of partworths

# Calculate the range for each attribute
importance = {}
importance['RAM'] = partworths[partworths['8GB_RAM'] == 1]['Rating'].mean() - partworths[partworths['16GB_RAM'] == 1]['Rating'].mean()
importance['Screen Size'] = partworths[partworths['14_inch_screen'] == 1]['Rating'].mean() - partworths[partworths['15_inch_screen'] == 1]['Rating'].mean()
importance['Battery Life'] = partworths[partworths['8_hour_battery'] == 1]['Rating'].mean() - partworths[partworths['10_hour_battery'] == 1]['Rating'].mean()
importance['Price'] = partworths[partworths['price_850'] == 1]['Rating'].mean() - partworths[partworths['price_900'] == 1]['Rating'].mean()
importance['Hard Drive'] = partworths[partworths['SATA_HardDrive'] == 1]['Rating'].mean() - partworths[partworths['SATA_HardDrive'] == 0]['Rating'].mean()

# Display the importance of each attribute
importance


{'RAM': 0.6833333333333336,
 'Screen Size': -0.6250000000000009,
 'Battery Life': -0.3683333333333314,
 'Price': 1.6524999999999999,
 'Hard Drive': -0.38928571428571423}

In [11]:
# Step 6: Determine the best specification for a laptop that maximizes consumer rating

# For each attribute, select the level with the highest average rating
best_specification = {
    'RAM': '16GB' if partworths[partworths['16GB_RAM'] == 1]['Rating'].mean() > partworths[partworths['8GB_RAM'] == 1]['Rating'].mean() else '8GB',
    'Screen Size': '15 inches' if partworths[partworths['15_inch_screen'] == 1]['Rating'].mean() > partworths[partworths['14_inch_screen'] == 1]['Rating'].mean() else '14 inches',
    'Battery Life': '10 hours' if partworths[partworths['10_hour_battery'] == 1]['Rating'].mean() > partworths[partworths['8_hour_battery'] == 1]['Rating'].mean() else '8 hours',
    'Price': '$850' if partworths[partworths['price_850'] == 1]['Rating'].mean() > partworths[partworths['price_900'] == 1]['Rating'].mean() else '$900',
    'Hard Drive': 'SSD' if partworths[partworths['SATA_HardDrive'] == 1]['Rating'].mean() > partworths[partworths['SATA_HardDrive'] == 0]['Rating'].mean() else 'HDD'
}

# Display the best specification for the laptop
best_specification

{'RAM': '8GB',
 'Screen Size': '15 inches',
 'Battery Life': '10 hours',
 'Price': '$850',
 'Hard Drive': 'HDD'}

In [13]:
# Step 7: Quantify the dollar value of specific attribute levels

# Determine the partworth difference for the price levels
price_diff = partworths[partworths['price_900'] == 1]['Rating'].mean() - partworths[partworths['price_850'] == 1]['Rating'].mean()

# Calculate dollar values for specified attribute levels based on price difference
dollar_values = {
    '16GB RAM': (partworths[partworths['16GB_RAM'] == 1]['Rating'].mean() - partworths[partworths['8GB_RAM'] == 1]['Rating'].mean()) / price_diff * 50,  # Assuming $50 increment between prices
    '15 inches screen': (partworths[partworths['15_inch_screen'] == 1]['Rating'].mean() - partworths[partworths['14_inch_screen'] == 1]['Rating'].mean()) / price_diff * 50,
    'SSD hard drive': (partworths[partworths['SATA_HardDrive'] == 1]['Rating'].mean() - partworths[partworths['SATA_HardDrive'] == 0]['Rating'].mean()) / price_diff * 50,
    '10 hours battery life': (partworths[partworths['10_hour_battery'] == 1]['Rating'].mean() - partworths[partworths['8_hour_battery'] == 1]['Rating'].mean()) / price_diff * 50
}

# Display the dollar values of each specified attribute level
dollar_values

{'16GB RAM': 20.67574382249118,
 '15 inches screen': -18.91074130105903,
 'SSD hard drive': 11.778690296088177,
 '10 hours battery life': -11.14473020675738}

In [15]:
# Step 8: Interpretation of Negative Dollar Values

# Screen Size: Calculate and compare the average ratings for 15 inches vs. 14 inches

# Calculate average rating for laptops with a 15-inch screen
avg_rating_screen_15 = partworths[partworths['15_inch_screen'] == 1]['Rating'].mean()

# Calculate average rating for laptops with a 14-inch screen
avg_rating_screen_14 = partworths[partworths['14_inch_screen'] == 1]['Rating'].mean()

# Calculate the difference in average ratings between 15 inches and 14 inches
screen_size_difference = avg_rating_screen_15 - avg_rating_screen_14

# Display the comparison for screen size
print("Average Rating for Screen Size (15 inches):", avg_rating_screen_15)
print("Average Rating for Screen Size (14 inches):", avg_rating_screen_14)
print("Difference (15 inches - 14 inches):", screen_size_difference)


# Battery Life: Calculate and compare the average ratings for 10 hours vs. 8 hours

# Calculate average rating for laptops with 10-hour battery life
avg_rating_battery_10 = partworths[partworths['10_hour_battery'] == 1]['Rating'].mean()

# Calculate average rating for laptops with 8-hour battery life
avg_rating_battery_8 = partworths[partworths['8_hour_battery'] == 1]['Rating'].mean()

# Calculate the difference in average ratings between 10 hours and 8 hours
battery_life_difference = avg_rating_battery_10 - avg_rating_battery_8

# Display the comparison for battery life
print("\nAverage Rating for Battery Life (10 hours):", avg_rating_battery_10)
print("Average Rating for Battery Life (8 hours):", avg_rating_battery_8)
print("Difference (10 hours - 8 hours):", battery_life_difference)

Average Rating for Screen Size (15 inches): 8.425
Average Rating for Screen Size (14 inches): 7.8
Difference (15 inches - 14 inches): 0.6250000000000009

Average Rating for Battery Life (10 hours): 7.959999999999999
Average Rating for Battery Life (8 hours): 7.591666666666668
Difference (10 hours - 8 hours): 0.3683333333333314
